# Displaying Sensor Data on the LED Bar

A key concept in computers is taking an input from a sensor, processing that data in a meaningful way, and outputting that to the user in a format that is intuitive and easy to read. This concept can be found in almost every computing system. 

We are going to create this type of system with our sensor and LED bar. The end product will be a sort of "digital thermometer".

### This notebook will walk you through using both the ATH20 sensor


As usual, apply the base overlay to the board:

In [1]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

# 1. Read Temperature Data

## ATH02 Sensor

If your sensor looks like this, it is an ATH20. If not, you should use the TH02 notebook [here](temp_ledbar_example_TH02.ipynb).

![ATH20](https://static-cdn.seeedstudio.site/media/catalog/product/cache/b2267b506d4e4594666ef83a79896a9a/1/0/101990644_4_.png)

### Install Driver

First, install the driver that will allow your board to communicate with this sensor (you  may already have downloaded it, but run this cell just in case).

In [2]:
!pip3 install git+https://github.com/LewisMcL/pynq_drivers --no-deps

  Cloning https://github.com/LewisMcL/pynq_drivers to /tmp/pip-wqhg2o6g-build


### Plug in your ATH20 Sensor

Connect you ATH20 Sensor to the port labeled G3 on your PMOD to Grove Adaptor (pictured below). Plug the adaptor into the port labeled PMODB on your PYNQ Board.

![pmod_adaptor](https://ce8dc832c.cloudimg.io/fit/640x480/n@a5b92b913168a98c8e5229bb3b0b33a39779b8b3/_cdn_/8F/91/A0/00/0/662008_1.jpg?mark_url=_tme-wrk_%2Ftme_new.png&mark_pos=center&mark_size=100pp)

### Import Libraries

Make sure your PMOD to Grove adapter is connected to the PMODB port. We are importing PMOD_GROVE_G3, because the AHT20 sensor is connected to the 3rd Grove adaptor port. If you have it plugged into G4, you will have to do `from pynq.lib.pmod import PMOD_GROVE_G4` instead. 

In [3]:
from grove import Grove_AHT20
from pynq.lib.pmod import PMOD_GROVE_G3

### 4. Receiving Measurements
We instantiate the sensor using the Grove_ATH20 constructor. This is very similar to a function, but it returns an object that represents our sensor that we can then use to call functions like get_temp().

In [5]:
py_ht = Grove_AHT20(base.PMODB, PMOD_GROVE_G3)

humidity = py_ht.get_humidity()
temp_c = py_ht.get_temp()
print('Humidity: ', float("{0:.2f}".format(humidity)), '%')
print('Temperature: ', float("{0:.2f}".format(temp_c)), '°C')

Humidity:  33.32 %
Temperature:  27.5 °C


# 2. Display on LED bar

### Setup:

Attached the arduino shield to the top of your board. Connect the LED bar to the G7 port on the Arduino shield. Connect the ATH20 to port G3 on the PMOD to Grove adaptor. Run the following code to initiate connection with the LED bar:

In [30]:
from pynq.lib.arduino import Grove_LEDbar
from pynq.lib.arduino import ARDUINO_GROVE_G7

ledbar = Grove_LEDbar(base.ARDUINO, ARDUINO_GROVE_G7)

### Reset the LED Bar

The default state of the LED could be all sorts of things. Let's turn it fully of so we have somewhere to work from.

In [31]:
# Turn the LED bar off
ledbar.reset()

### Decide on Your Temperature Scale

We want to make a visual thermometer that displays the temperature across a range that we care about. Perhaps this is simply comfortable room temperature (somewhere between 60 and 80 degrees fahrenheit), or it's the temperature of industrial equipment that cannot get above 150 degrees fahrenheit. What we will do next is establish this range and "map" it to the 10 bars on the LED indicator, with the red being the highest end of the range and the green being the lowest. For this example, lets the temperature we might find in a house in Colorado, 60 to 80 degrees fahrenheit, or about 15 to 25 degrees celsius (remember, our sensors read celsius, so make sure to convert when necessary)

In [32]:
max_temp = 25
min_temp = 15
temp_range = max_temp - min_temp

We should consider "edge cases" where our temperature falls above or below our range. These are always good to check before we proceed. In this situation, we will illuminate all LEDs if the temperature is above the range, and 1 LED if it is below the range.

Once we have ruled out edge cases, we can deal with the case where our temperature falls within our allotted range. Because there are 10 bars, we can only only display the temperature in tenths of our range. Knowing what our range is, we can calculate a number between 0 and 10 to represent where within the range we fall. Note that we multiply by 10 and round to the nearest whole number to make sure we pass an integer between 0 and 10 to the function up_to().

Because our LED bar speaks in binary, We are going to use a function "up_to" which produces the binary value required to light up the bars "up to" a certain index on the led bar, given that index. This is covered more in depth in the [LED Bar notebook](../bootcamp_sessions/3_led_bar.ipynb).

In [33]:
def up_to(index):
    return ~(2**(10 - index) - 1)

In [34]:
# if temperature is above our range of measurement
if temp_c > max_temp:
    ledbar.write_binary(up_to(10))
# if temperature is below our range of measurement
elif temp_c < min_temp:
    ledbar.write_binary(up_to(0))
# otherwise is falls within our range
else:
    temp_index = int(round(((temp_c - min_temp)/temp_range)*10))
    ledbar.write_binary(up_to(temp_index))